In [1]:
import import_ipynb
from nexoheader import *
#import uproot 
import uproot3
import awkward as ak

importing Jupyter notebook from nexoheader.ipynb


In [2]:
tgt="Tl208" 
seed=55
nevts=5000


file = "/home/wrkshp/nexo/slurm_data/recon/ExternalGammas_light-%s-%i-seed%i_sensrecon.root"%(tgt,nevts,seed)
newfilename = "/home/wrkshp/nexo/slurm_data/sens/%s-seed%i.root"%(tgt,seed)




In [7]:
file.find("shp")

9

In [2]:

def simpletree(file="",newfilename=""):
    Treenames=["Energy","Standoff","ChargeQuanta","Photons","DNNTag","NESTBugFlag"]
    
    
    
    
    rootfile=uproot.open(file)
    
    
    BR=1.0
    if file.find("Tl208") >= 1:
        BR=0.36
    
    
    #newrootfile=uproot.recreate(newfile)
    trees=[]

    treesinfo=[]
    bdicts={}
    b_dicts_info={}
    branches=[]
    bb=0

    for i,tname in enumerate(Treenames):
      #  print(len(trees))
        treeinfo={}
        
        try:
            trees.append(rootfile["Event/Recon/%s/%s"%(tname,tname)])
            treeinfo["name"]=tname
            treeinfo["path"]="Event/Recon/%s/%s"%(tname,tname)
            treeinfo["branch"]=rootfile["Event/Recon/%s/%s/%s"%(tname,tname,tname)]
        except:
            print("Issue with ",tname, " on file ",file)      
            return 0   
        print(treeinfo["branch"].keys())
        if True: 
            for j, bname in enumerate(treeinfo["branch"].keys()):
                if j < 4:
                    continue 
                #print(bname)
                bdict={}
                bdict["name"]=bname
                bdict["values"]=treeinfo["branch"][bname].array()

                branches.append(rootfile["Event/Recon/%s/%s/%s"%(tname,tname,bname)])
                #bdicts.append(bdict)
                bdicts[bname]=treeinfo["branch"][bname].array()

                b_dicts_info[bname]=type(bdict["values"][0])
                #print("\t\t\t\t ",len(bdicts),"  ",len(b_dicts_info))
    
    bdicts["Branch_ratio"]= ak.highlevel.Array([BR]*len(bdicts[bname]))
  
    b_dicts_info["Branch_ratio"]= type(bdict["values"][0])        
    with uproot3.recreate(newfilename) as f:
        f["nexo"] = uproot3.newtree(b_dicts_info,title="")

        f["nexo"].extend(bdicts)
    
    f.close()
    
    
    
    
    return f["nexo"]

In [3]:
FI=file_dict_from_file("/home/wrkshp/nexo/slurm_data/Tl208/ExternalGammas_light-v2-Tl208-5000-seed53.root")

FI
rf =uproot.open(FI["root_filename"])





#rf.close()

rf.keys()
#ExternalGammas_light-v2-Tl208-5000-seed53.root

rf["Meta/FileMetaData"]

NameError: name 'uproot' is not defined

In [24]:
if 1==1:
    
#def simplecombtree(file="",newfilename=""):
    #file ="/home/wrkshp/nexo/slurm_data/Tl208/ExternalGammas_light-v3-Tl208-5000-seed20.root"
    newfilename="./data/Bi212_v3_seed10.root"
    file="./data/ExternalGammas_light-v3-Bi212-5000-seed10.root"
    sens_Treenames=["Event/Recon/Energy","Event/Recon/Standoff"
                   ,"Event/Recon/ChargeQuanta","Event/Recon/Photons"
                   ,"Event/Recon/DNNTag","Event/Recon/NESTBugFlag"]
    root_Treenames=["Event/Sim/SimEvent"]
    root_bnames=["fGenX"
                ,"fGenY"
                ,"fGenZ"
                ]
    senstreelen=len(sens_Treenames)
    
    Treenames=sens_Treenames+root_Treenames
    
    sens_Tree_basename ="Event/Recon/"
    root_Tree_basename ="Event/Sim/"
    
    Tree_basename=[sens_Tree_basename,root_Tree_basename]
    
    
    filedict=file_dict_from_file(file)
    print(filedict)
    Seed=filedict["Seed"]
    Nevts=filedict["NumEvents"]
    
    
    
    BR=1.0
    if file.find("Tl208") >= 1:
        BR=0.3594
    
    
    #newrootfile=uproot.recreate(newfile)
    trees=[]

    treesinfo=[]
    bdicts={}
    b_dicts_info={}
    branches=[]
    bb=0

    for i,tname in enumerate(Treenames):
        print(tname)
        if i < senstreelen:
            base_sel=0
        else:
            base_sel=1
        
        
        branchname=tname.replace(Tree_basename[base_sel],"")
        
        filename=""
        if i==0:
            filename=filedict["sens_filename"]
            rootfile=uproot3.open(filename)
            print("opening, ",filename)
            print("keys",rootfile.keys())
            
            
        elif i <senstreelen:
            keys=rootfile.keys()

            print(tname, keys)
            if tname+";1" in keys:
                #print("Good")
                aaa="AAA"
            else:
                #print(sens_Treenames)
                #print(senstreelen)
                print(tname,"Not in keys","issue with file and branch")
                #print(branchname, filename)
        
        else:
            try:
                keys=rootfile.keys()
                #print("\t\t",branchname, "\t",filename)
                if tname+";1" in keys:
                    #print(tname, " is in file")
                    aaa="AAA"
                else:
                    #print("Openning new file, end of sens")
                    filename=filedict["root_filename"]
                    rootfile.close()
                    #print(filename)
                    rootfile=uproot3.open(filename)
                    #print(rootfile.keys())
            except:    
                #print("Openning new file due to try")
                filename=filedict["root_filename"]
                rootfile=uproot3.open(filename)
            
            #filename=filedict["sens_filename"]
            #rootfile=uproot.open(filename)
        
      #  print(len(trees))
        treeinfo={}
        
        if base_sel==1:
            basepath="%s"%(tname)
        else:
            basepath="%s/%s"%(tname,branchname)
        
        #continue
        #print(basepath)
        try:
            trees.append(rootfile["%s/%s"%(tname,branchname)])
            treeinfo["name"]=branchname
            treeinfo["path"]="%s"%(basepath)
            treeinfo["branch"]=rootfile["%s/%s"%(basepath,branchname)]
        except:
            print("Issue with ",tname, " on file ",file) 
            
            #return 0   
        #print("%s/%s"%(basepath,branchname))
        #print(treeinfo["branch"].keys())
        if True: 
            try :
                enumerate(treeinfo["branch"].keys())
                #print("Looking good")
            except :
                print("Error")
                continue
            
            
            for j, bname in enumerate(treeinfo["branch"].keys()):
                if j < 4:
                    continue 
                
                if base_sel==1:
                    if bname not in root_bnames:
                        #print("Skipping: not wantted")
                        continue
                
                
                #print("bname",bname)    
                bdict={}
                bdict["name"]=bname
                bdict["values"]=treeinfo["branch"][bname].array()
                bnamepath="%s/%s/%s"%(basepath,branchname,bname)
                #print(bnamepath,rootfile["%s/%s"%(basepath,branchname)].keys())
                #print(rootfile["Event/Recon/Standoff/Standoff/Standoff/standoff"].array())
                #print(bname)

                #branches.append(rootfile[bnamepath])
                #bdicts.append(bdict)
                bdicts[bname]=treeinfo["branch"][bname].array()
                entries=len(bdicts[bname])
                b_dicts_info[bname]=type(bdict["values"][0])
                #print("\t\t\t\t ",len(bdicts),"  ",len(b_dicts_info))
    #print("Done with branches")
    
    bdicts["Branch_ratio"]= ak.highlevel.Array([BR]*(entries))
    bdicts["Seed"]= ak.highlevel.Array([Seed]*(entries))
    bdicts["Nevts"]= ak.highlevel.Array([Nevts]*(entries))
  
    b_dicts_info["Branch_ratio"]= type(bdict["values"][0]) 
    b_dicts_info["Seed"]= type(bdict["values"][0]) 
    b_dicts_info["Nevts"]= type(bdict["values"][0]) 
    with uproot3.recreate(newfilename) as f:
        f["nexo"] = uproot3.newtree(b_dicts_info,title="")

        f["nexo"].extend(bdicts)
    
    f.close()
    
    
    
    
    #return f["nexo"]

{'Config': 'ExternalGammas_light', 'Target': 'Bi212', 'NumEvents': 5000, 'Seed': '10', 'Version': 'v3', 'root_filename': './data/ExternalGammas_light-v3-Bi212-5000-seed10.root', 'sens_filename': './data/ExternalGammas_light-v3-Bi212-5000-seed10_sensrecon.root'}
Event/Recon/Energy
opening,  ./data/ExternalGammas_light-v3-Bi212-5000-seed10_sensrecon.root
keys [b'Event;1', b'ProcessID0;1']
Event/Recon/Standoff
Event/Recon/Standoff [b'Event;1', b'ProcessID0;1']
Event/Recon/Standoff Not in keys issue with file and branch
Event/Recon/ChargeQuanta
Event/Recon/ChargeQuanta [b'Event;1', b'ProcessID0;1']
Event/Recon/ChargeQuanta Not in keys issue with file and branch
Event/Recon/Photons
Event/Recon/Photons [b'Event;1', b'ProcessID0;1']
Event/Recon/Photons Not in keys issue with file and branch
Event/Recon/DNNTag
Event/Recon/DNNTag [b'Event;1', b'ProcessID0;1']
Event/Recon/DNNTag Not in keys issue with file and branch
Event/Recon/NESTBugFlag
Event/Recon/NESTBugFlag [b'Event;1', b'ProcessID0;1']
E

In [58]:
if 1==1:
    
#def simplecombtree(file="",newfilename=""):
    #file ="/home/wrkshp/nexo/slurm_data/Tl208/ExternalGammas_light-v3-Tl208-5000-seed20.root"
    newfilename="./data/Bi212_v3_seed10.root"
    file="./data/ExternalGammas_light-v3-Bi212-5000-seed10.root"
    sens_Treenames=["Event/Recon/Energy","Event/Recon/Standoff"
                   ,"Event/Recon/ChargeQuanta","Event/Recon/Photons"
                   ,"Event/Recon/DNNTag","Event/Recon/NESTBugFlag"]
    root_Treenames=["Event/Sim/SimEvent"]
    root_bnames=["fGenX"
                ,"fGenY"
                ,"fGenZ"
                ]
    senstreelen=len(sens_Treenames)
    
    Treenames=sens_Treenames+root_Treenames
    
    sens_Tree_basename ="Event/Recon/"
    root_Tree_basename ="Event/Sim/"
    
    Tree_basename=[sens_Tree_basename,root_Tree_basename]
    
    
    filedict=file_dict_from_file(file)
    print(filedict)
    Seed=filedict["Seed"]
    Nevts=filedict["NumEvents"]
    
    
    
    BR=1.0
    if file.find("Tl208") >= 1:
        BR=0.3594
    
    
    #newrootfile=uproot.recreate(newfile)
    trees=[]

    treesinfo=[]
    bdicts={}
    b_dicts_info={}
    branches=[]
    bb=0

    for i,tname in enumerate(Treenames):
        print(i,tname)
        if i < senstreelen:
            base_sel=0
        else:
            base_sel=1
        
        
        branchname=tname.replace(Tree_basename[base_sel],"")
        
        filename=""
        if i==0:
            filename=filedict["sens_filename"]
            rootfile=uproot3.open(filename)
            print("  opening, ",filename)
            print("  keys",rootfile.keys())
            
            
        elif i <senstreelen:
            keys=rootfile.keys()

            print(tname, keys)
            if tname+";1" in keys:
                #print("Good")
                aaa="AAA"
            else:
                #print(sens_Treenames)
                #print(senstreelen)
                print("  ",tname,"Not in keys","issue with file and branch")
                #print(branchname, filename)
        
        else:
            try:
                keys=rootfile.keys()
                #print("\t\t",branchname, "\t",filename)
                if tname+";1" in keys:
                    #print(tname, " is in file")
                    aaa="AAA"
                else:
                    #print("Openning new file, end of sens")
                    filename=filedict["root_filename"]
                    rootfile.close()
                    #print(filename)
                    rootfile=uproot3.open(filename)
                    #print(rootfile.keys())
            except:    
                #print("Openning new file due to try")
                filename=filedict["root_filename"]
                rootfile=uproot3.open(filename)
            
            #filename=filedict["sens_filename"]
            #rootfile=uproot.open(filename)
        
      #  print(len(trees))
        treeinfo={}
        
        if base_sel==1:
            basepath="%s"%(tname)
        else:
            basepath="%s/%s"%(tname,branchname)
        
        #continue
        #print(basepath)
        try:
            trees.append(rootfile["%s/%s"%(tname,branchname)])
            treeinfo["name"]=branchname
            treeinfo["path"]="%s"%(basepath)
            treeinfo["branch"]=rootfile["%s/%s"%(basepath,branchname)]
        except:
            print("Issue with ",tname, " on file ",file) 
            
            #return 0   
        #print("%s/%s"%(basepath,branchname))
        #print(treeinfo["branch"].keys())
        if True: 
            try :
                enumerate(treeinfo["branch"].keys())
                #print("Looking good")
            except :
                print("Error")
                continue
            
            
            for j, bname in enumerate(treeinfo["branch"].keys()):
                #print("\tj loop", j, bname,len(bdicts.keys()))
                if b"nEXO::EventObject" == bname:
                    continue 
                
                if base_sel==1:
                    if bname.decode("utf-8") not in root_bnames:
                        #print("\t\tSkipping: not wantted", bname)
                        
                        continue
                
                
                #print("bname",bname)    
                bdict={}
                bdict["name"]=bname
                bdict["values"]=treeinfo["branch"][bname].array()
                bnamepath="%s/%s/%s"%(basepath,branchname,bname)
                #print(bnamepath,rootfile["%s/%s"%(basepath,branchname)].keys())
                #print(rootfile["Event/Recon/Standoff/Standoff/Standoff/standoff"].array())
                #print("\tBuild dict in j loop",bname)

                #branches.append(rootfile[bnamepath])
                #bdicts.append(bdict)
                bdicts[bname]=treeinfo["branch"][bname].array()
                entries=len(bdicts[bname])
                b_dicts_info[bname]=type(bdict["values"][0])
                #print("\t\t\t\t ",len(bdicts),"  ",len(b_dicts_info))
    #print("Done with branches")
    
    bdicts["Branch_ratio"]= ak.highlevel.Array([BR]*(entries))
    bdicts["Seed"]= ak.highlevel.Array([Seed]*(entries))
    bdicts["Nevts"]= ak.highlevel.Array([Nevts]*(entries))
  
    b_dicts_info["Branch_ratio"]= type(bdict["values"][0]) 
    b_dicts_info["Seed"]= type(bdict["values"][0]) 
    b_dicts_info["Nevts"]= type(bdict["values"][0]) 
    with uproot3.recreate(newfilename) as f:
        for k,name in enumerate(b_dicts_info.keys()):
            print(name)
            f["nexo"] = uproot3.newtree({name:b_dicts_info[name]})
            f["nexo"].extend({name:bdicts[name]})



        #f["nexo"] = uproot3.newtree(b_dicts_info,title="")

        #f["nexo"].extend(bdicts)
    
    f.close()
    
    
    
    
    #return f["nexo"]

{'Config': 'ExternalGammas_light', 'Target': 'Bi212', 'NumEvents': 5000, 'Seed': '10', 'Version': 'v3', 'root_filename': './data/ExternalGammas_light-v3-Bi212-5000-seed10.root', 'sens_filename': './data/ExternalGammas_light-v3-Bi212-5000-seed10_sensrecon.root'}
0 Event/Recon/Energy
  opening,  ./data/ExternalGammas_light-v3-Bi212-5000-seed10_sensrecon.root
  keys [b'Event;1', b'ProcessID0;1']
1 Event/Recon/Standoff
Event/Recon/Standoff [b'Event;1', b'ProcessID0;1']
   Event/Recon/Standoff Not in keys issue with file and branch
2 Event/Recon/ChargeQuanta
Event/Recon/ChargeQuanta [b'Event;1', b'ProcessID0;1']
   Event/Recon/ChargeQuanta Not in keys issue with file and branch
3 Event/Recon/Photons
Event/Recon/Photons [b'Event;1', b'ProcessID0;1']
   Event/Recon/Photons Not in keys issue with file and branch
4 Event/Recon/DNNTag
Event/Recon/DNNTag [b'Event;1', b'ProcessID0;1']
   Event/Recon/DNNTag Not in keys issue with file and branch
5 Event/Recon/NESTBugFlag
Event/Recon/NESTBugFlag [b'

NotImplementedError: 

In [74]:
with uproot3.recreate(newfilename) as f:
    for k,name in enumerate(b_dicts_info.keys()):
        
        try:
            f["nexo"] = uproot3.newtree({name:b_dicts_info[name]})
            f["nexo"].extend({name:bdicts[name]})
        except:
            print("issue",name,len(bdicts[name]))

TypeError: unhashable type: 'slice'

In [84]:
bdicts[b'energy_mctruth_inTPC']



array([0.3709306 , 0.39421842, 0.7025029 , 0.5092402 , 1.3048906 ,
       1.6207314 , 0.17192516, 1.6207385 , 0.38819075, 0.4846769 ,
       0.7273287 , 0.39293885, 0.8155838 , 0.7273285 , 0.41649836,
       0.319727  , 0.7853685 , 0.41957197, 1.0786271 , 0.35070887,
       0.4106513 , 0.283091  , 0.7273356 , 1.6207311 , 0.5326867 ,
       0.18321936, 1.459611  , 1.0425417 , 1.2496232 , 0.51756793,
       0.6733718 , 0.7151797 ], dtype=float32)

In [35]:
rootfile["Event/Recon/DNNTag/DNNTag/DNNTag/m_DNNvalue"].array()

array([0.40363258, 0.01079542, 0.01955967, 0.1360321 , 0.04920904,
       0.14081533, 0.06496177, 0.10722533, 0.03951696, 0.0174946 ,
       0.01247369, 0.06268373, 0.27959961, 0.01014541, 0.01255884,
       0.09864623, 0.01999658, 0.01959733, 0.11189628, 0.02292522,
       0.16788332, 0.02749834, 0.08434334, 0.0696657 , 0.03819784,
       0.04620985, 0.05869805, 0.22834928, 0.06454292, 0.00853588,
       0.01675245, 0.03697034, 0.0852848 , 0.32578176, 0.03611074,
       0.03652194, 0.08436024, 0.11792047, 0.08290456, 0.08182105,
       0.0311686 , 0.00802624])

In [57]:
#tgt="Th228"
version="-v3"
target_chain=["Tl208"]#,"Tl208","Ra224","Pb212","Bi212"]

#target_chain=["Ra226","Pb214", "Bi214", "Bi210", "Pb206"]



#target_chain=["Ra224"]
#target_chain=["Pb212"]
target_chain=["Bi212"]
#target_chain=["Cs137"]
#target_chain=["Co60"]
#target_chain=["Th228"]
#target_chain=["Pb206"]
#seed=55
nevts=5000
repeat=True
#repeat=False

#tgt="Tl208"

#repeat=True
#seed =55


Missing_list=[]

#file = "/home/wrkshp/nexo/slurm_data/recon/ExternalGammas_light-%s-%i-seed%i_sensrecon.root"%(tgt,nevts,seed)
#newfile = "/home/wrkshp/nexo/slurm_data/sens/%s-seed%i_sens.root"%(tgt,seed)
#Fullfilename="/home/wrkshp/nexo/slurm_data/sens/%s-sens.root"%(tgt)

#with uproot3.recreate(Fullfilename) as Fullfile:
for tgt in target_chain:
    if True  :  

        for seed in range(1,2+1,1):
            mainpath="/Users/EXO_Lab1/Documents/Bane/nexo_code/nexo/Ext_src_cal"
            file = mainpath+"/data/ExternalGammas_light%s-%s-%i-seed%i.root"%(version,tgt,nevts,seed)            
            filerec = mainpath+"/data/ExternalGammas_light%s-%s-%i-seed%i_sensrecon.root"%(version,tgt,nevts,seed)            
            print(file)
            newfile = "./data/%s%s-seed%i.root"%(tgt,version,seed)

            #file = "/home/wrkshp/nexo/slurm_data/%s/ExternalGammas_light%s-%s-%i-seed%i.root"%(tgt,version,tgt,nevts,seed)            
            #filerec = "/home/wrkshp/nexo/slurm_data/recon/ExternalGammas_light%s-%s-%i-seed%i_sensrecon.root"%(version,tgt,nevts,seed)            
            #print(file)
            #newfile = "/home/wrkshp/nexo/slurm_data/sens/%s%s-seed%i.root"%(tgt,version,seed)
            Missing={}
            if os.path.exists(newfile):
                if not repeat:
                    print("Skipping seed",seed)
                    continue

            if os.path.exists(file) and os.path.exists(filerec) :
                
                T=simplecombtree(file,newfile)
                if T==0:
                    print("Error with ",seed)
                    Missing["Target"]=tgt
                    Missing["Seed"]=seed
                    Missing_list.append(Missing)
                else:
                    print("Finished ,", seed)
            else:
                print(tgt, version,seed ," is missing")
                Missing["Target"]=tgt
                Missing["Seed"]=seed
                
                Missing_list.append(Missing)

Missing_df=pd.DataFrame.from_dict(Missing_list)                
                

            
#Fullfile.close()            

/Users/EXO_Lab1/Documents/Bane/nexo_code/nexo/Ext_src_cal/data/ExternalGammas_light-v3-Bi212-5000-seed1.root


NameError: name 'simplecombtree' is not defined

In [15]:
print(len(Missing_list))
try:
    Missing_df_Pb=Missing_df.query("Target == 'Tl208'")
    Missing_df_Pb
except:
    print("No missing?")

64


In [14]:
 #def missingstring(misslist,cuts):
if 1:
    misslist=Missing_df
    cuts="Target == 'Tl208'"
    DF=misslist.query(cuts)
    seeds = ""
    for i in DF["Seed"]:
        seeds = seeds+",%i"%i
    seeds=seeds[1:]
    print (seeds)
    #return seeds
    

In [ ]:
ExternalGammas_light-v2-Tl208-5000-seed1_sensrecon.root

'./data/ExternalGammas_light-v3-Bi212-5000-seed10.root'

In [ ]:
branches


In [ ]:
type(rootfile["Event/Recon/%s/%s"%(tname,tname)])

In [ ]:
tt=uproot.TTree
tt

In [ ]:
#newfile["T"]


#newfile["T"].extend(bdicts)
tt.name="T"
tt.name

tt.branches=bdicts
tt.branches

In [ ]:
newrootfile=uproot.recreate(newfile)




In [ ]:
type(tt.branches)

In [ ]:
newrootfile["T"]=tt

In [ ]:
t=rootfile["Event/Recon/%s/%s"%(tname,tname)]

In [ ]:
'DNNTag/m_DNNvalue't['DNNTag/m_DNNvalue'].array()

In [ ]:
T=uproot.TTree
print(T.name,"\n",
T.title,"\n",
T.branches
     )

In [ ]:
T.name="T"
T.branches=branches

In [ ]:
T.branches

In [ ]:
newfile

In [ ]:
import uproot3

In [ ]:
bdicts

In [ ]:
nfile=uproot3.recreate("tmp.root", compression=uproot3.ZLIB(4))
nfile["t"]=uproot3.newtree(b_dicts_info )
nfile["t"].extend(bdicts)      


In [ ]:
newtree = uproot3.newtree(b_dicts_info)


In [ ]:
type(bdicts)

In [ ]:
with uproot3.recreate("example.root") as f:
    f["t"] = uproot3.newtree(b_dicts_info,title="TTree title")
    #for i,b in enumerate(bdicts):
    #f["t"].newbranch()
    f["t"].extend(bdicts)
f.close()

In [ ]:
f

In [ ]:
50*5000

In [ ]:
f['t/energy'].array()

In [ ]:
len(bdicts)

In [ ]:
rfile=uproot.open("/home/wrkshp/nexo/slurm_data/sens/Co60-seed1.root")

In [ ]:
rfile.keys()

In [ ]:
tgt="Tl208"
version="-v2"
nevts=5000
seed=55

file = "/home/wrkshp/nexo/slurm_data/Tl208/ExternalGammas_light%s-%s-%i-seed%i.root"%(version,tgt,nevts,seed)
newfilename = "/home/wrkshp/nexo/slurm_data/sens/%s%s-seed%i.root"%(tgt,version,seed)


rfile=uproot.open(file)

In [ ]:
rfile["Event/Sim/SimEvent/fGenParticleID"].values()

In [ ]:
Energy_hist,Energy_bin_edges=np.histogram(
        np.array(T["energy"].array()),bins=nbins,range=binrange)




